In [1]:
import os
import sys
import pickle
import numpy as np

In [137]:
def load_batch(fpath, label_key='labels'):
    """Internal utility for parsing CIFAR data.
    # Arguments
        fpath: path the file to parse.
        label_key: key for label data in the retrieve
            dictionary.
    # Returns
        A tuple `(data, labels)`.
    """
    with open(fpath, 'rb') as f:
        if sys.version_info < (3,):
            d = pickle.load(f)
        else:
            d = pickle.load(f, encoding='bytes')
            # decode utf8
            d_decoded = {}
            for k, v in d.items():
                d_decoded[k.decode('utf8')] = v
            d = d_decoded
    data = d['data']
    labels = d[label_key]

    #data = data.reshape(data.shape[0], 3, 32, 32)
    data = data.reshape(data.shape[0], 3072)
    return np.array(data), np.array(labels)

In [138]:
path = 'cifar-10-batches-py'
train_fpath = os.path.join('../', path, 'data_batch_1')
val_fpath = os.path.join('../', path, 'data_batch_2')
test_fpath = os.path.join('../', path, 'test_batch')

x_train, y_train = load_batch(train_fpath)
x_val, y_val = load_batch(val_fpath)
x_test, y_test = load_batch(test_fpath)

print('x_train shape:\t', x_train.shape, '\t y_train shape:\t', y_train.shape)
print('x_val shape:\t', x_val.shape, '\t y_val shape:\t', y_val.shape)
print('x_test shape:\t', x_test.shape, '\t y_test shape:\t', y_test.shape)

x_train shape:	 (10000, 3072) 	 y_train shape:	 (10000,)
x_val shape:	 (10000, 3072) 	 y_val shape:	 (10000,)
x_test shape:	 (10000, 3072) 	 y_test shape:	 (10000,)


In [139]:
mean, std = x_train.mean(axis=0), x_train.std(axis=0)
print('mean shape:\t', mean.shape, '\nstd shape:\t', std.shape)

x_train = ( x_train - mean ) / std
x_val = ( x_val- mean ) / std
x_test = ( x_test - mean ) / std

print(x_train.mean(), x_train.std())
print(x_val.mean(), x_val.std())
print(x_test.mean(), x_test.std())

mean shape:	 (3072,) 
std shape:	 (3072,)
2.2611542268199023e-18 0.9999999999999997
-0.002367710988733489 0.9987174761512392
0.007943731845370715 0.995771074950348


In [255]:
K = 10  # number of classes

W = np.random.normal(0, 0.01, (K, 3072))
b = np.random.normal(0, 0.01, K)

In [238]:
def evaluate_classifier(X, W, b):
    P = np.zeros((X.shape[0], K))
    
    for i in range(X.shape[0]):
        P[i] = np.dot(W, X[0]) + b
        
    return np.exp(P) / np.sum(np.exp(P), axis=0)

In [239]:
P = evaluate_classifier(x_train[:5], W, b)

In [262]:
def compute_cost(X, Y, W, b, lamb):
    
    cost = 0

    P = evaluate_classifier(X, W, b)
    
    for i in range(X.shape[0]):
        cost += -np.log2(P[i][Y[i]] + sys.float_info.epsilon)
        
    cost /=  X.shape[0]
    
    cost += lamb * np.sum(W**2)
    
    return cost

In [266]:
lamb = 0.3
# esto no chuta porque a mas samples mas cost y se esta diviendo por X.shape[0] asi que mala cosa
compute_cost(x_train[:2000], y_train[:2000], W, b, lamb)

11.894097624288035